✅ Step 1: Load Your Classifier and Make a Prediction

In [1]:
import pickle
import numpy as np

# Load trained model
with open("rf_classifier.pkl", "rb") as f:
    classifier = pickle.load(f)

# Sample patient data (replace with actual input)
patient_data = {
    "age": 57,
    "sex": 1,
    "cp": 2,
    "trestbps": 140,
    "chol": 250,
    "fbs": 0,
    "restecg": 1,
    "thalach": 150,
    "exang": 0,
    "oldpeak": 1.4,
    "slope": 2,
    "ca": 0,
    "thal": 2
}

# Ensure feature order matches training
features = np.array([list(patient_data.values())]).reshape(1, -1)

# Get prediction
prediction = classifier.predict(features)[0]  # 0 = No Heart Disease, 1 = Likely Heart Disease
prediction

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


np.int64(1)

In [10]:
!pip install transformers accelerate -q

In [4]:
!pip install sacremoses -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 47.6 MB/s eta 0:00:00


✅ Step 2: Install and Load Mistral




In [19]:
from huggingface_hub import login
login()

In [24]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Step 3: Build Prompt for the LLM

In [27]:
def generate_prompt(patient_data, prediction):
    label = "Heart disease likely" if prediction == 1 else "No heart disease likely"

    instruction = f"""### Instruction:
You are a friendly and knowledgeable medical assistant. A patient has entered their health data into the system.

Based on the analysis, the system predicts: **{label}**

Please respond to the patient in simple, clear language. Start by acknowledging the data, explain what the prediction means, and guide them on what they should do next. Be supportive and avoid technical jargon.

### Response:"""

    return instruction


In [28]:
prompt = generate_prompt(patient_data, prediction)

response = generator(prompt, max_new_tokens=200, do_sample=True, temperature=0.7)
print(response[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### Instruction:
You are a friendly and knowledgeable medical assistant. A patient has entered their health data into the system.

Based on the analysis, the system predicts: **Heart disease likely**

Please respond to the patient in simple, clear language. Start by acknowledging the data, explain what the prediction means, and guide them on what they should do next. Be supportive and avoid technical jargon.

### Response:

Hello! Thank you for entering your health data into our system. I'm sorry to say that based on the information you've provided, our system predicts that you are at a higher risk for heart disease. This doesn't mean you will definitely develop heart disease, but it's important to take this prediction seriously and take steps to manage your heart health.

There are several things you can do to help reduce your risk of heart disease. First, it's important to maintain a healthy diet that is low in saturated and trans fats, high in fiber, and includes plenty of fruits an